<a href="https://colab.research.google.com/github/Kalyankr/multi-class-classification-and-model-comparision-/blob/master/Multi_class_text_classification_modelcomparision_%26_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
%matplotlib inline

In [0]:
!wget https://storage.googleapis.com/tensorflow-workshop-examples/stack-overflow-data.csv

In [0]:
data=pd.read_csv("../content/stack-overflow-data.csv")

In [0]:
data.head()

Will search for any Null post and tags. if prsent will drop them.

In [0]:
print(data["tags"].isnull().sum())
print(data["post"].isnull().sum())

lets see how length "post" column looks exactly in the data 

In [0]:
print(data["post"].apply(lambda x:len(x.split(' '))).sum())

lets see the distribution of the labels in the data

In [0]:
unique_tags=data["tags"].unique()
unique_tags


In [0]:
plt.figure(figsize=(10,4))
data.tags.value_counts().plot(kind="bar")

looks like data distrubution was well balanced

In [0]:
#look a few post and tag pairs.

def print_plt(index):
  example=data[data.index==index][["post",'tags']].values[0]
  if len(example) >0:
    print(example[0])
    print("Tag:",example[1])
print_plt(10)

we need to remove the Html tags, puntuations and stogwords

In [0]:
replace_by_space=re.compile("[/(){}\[\]\|@,;]")
bad_symbols=re.compile('[^0-9a-z #+_]')
Stopwords=set(stopwords.words('english'))

In [0]:
def clean_text(text):
  text=BeautifulSoup(text,"lxml").text # HTML decoding
  text=text.lower() # lowercase text
  text=replace_by_space.sub(" ",text) # replace symbols by space in text
  text=bad_symbols.sub("",text) # delete sysmbols which are in bad_symbols from text
  text=" ".join(word for word in text.split() if word not in Stopwords) # remoce stop words
  return  text


data["post"]=data["post"].apply(clean_text)
print_plt(10)

In [0]:
data["post"].apply(lambda x: len(x.split(" "))).sum()

In [0]:
# train and test sets
x=data.post
y=data.tags
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [0]:
x_train.shape,y_train.shape

lets first go through Navie Bayes Classifier for multinomial model

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

nb=Pipeline([("vect",CountVectorizer()),
            ("tfidf",TfidfTransformer()),
            ("clf",MultinomialNB())])
nb.fit(x_train,y_train)

In [0]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
y_pred= nb.predict(x_test)

In [0]:
print("Accuracy %s" %accuracy_score(y_pred,y_test))
print(classification_report(y_test,y_pred,target_names=unique_tags))

thats great naive bayes give 75% accuracy

Linear Support Vector Machine is widely regarded as one of the best text classification algorithms.

In [0]:
from sklearn.linear_model import SGDClassifier

sgd=Pipeline([("vect",CountVectorizer()),
            ("tfidf",TfidfTransformer()),
            ("clf",SGDClassifier())])
sgd.fit(x_train,y_train)

In [0]:
y_pred= sgd.predict(x_test)
print("Accuracy %s" %accuracy_score(y_pred,y_test))
print(classification_report(y_test,y_pred,target_names=unique_tags))

Great we went upto 80% now with 5% increase than Naive bayes

Lets go with **Logistic Regression**

In [0]:
from sklearn.linear_model import LogisticRegression

lreg=Pipeline([("vect",CountVectorizer()),
            ("tfidf",TfidfTransformer()),
            ("clf",LogisticRegression())])
lreg.fit(x_train,y_train)

In [0]:
y_pred= lreg.predict(x_test)
print("Accuracy %s" %accuracy_score(y_pred,y_test))
print(classification_report(y_test,y_pred,target_names=unique_tags))

we got around 80%accuracy with logistic regression

we use Word2Vec and logistic regression. 

In [0]:
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [0]:
from itertools import islice
list(islice(wv.vocab, 13030, 13050))

In [0]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
return np.vstack([word_averaging(wv, post) for post in text_list ])

In [0]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [0]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['tags'])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test.tags))
print(classification_report(test.tags, y_pred,target_names=my_tags))

we got around 63% accuracy

Doc2vec and Logistic Regression

you can find gensim doc2vec here https://github.com/RaRe-Technologies/gensim/blob/ca0dcaa1eca8b1764f6456adac5719309e0d8e6d/docs/notebooks/doc2vec-IMDB.ipynb

In [0]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled
X_train, X_test, y_train, y_test = train_test_split(data.post, data.tags, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [0]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
model_dbow.min_alpha = model_dbow.alpha

In [0]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [0]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

we got around 80%

Lets use BOW with Keras 

In [0]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(data) * .7)
train_posts = data['post'][:train_size]
train_tags = data['tags'][:train_size]

test_posts = data['post'][train_size:]
test_tags = data['tags'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 3

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
validation_split=0.1)

In [0]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

now we got an accuracy of 79% 